#   8.4 潜在ディリクレ配分によるトピックモデルの構築

## トピックモデル 　
ニュースなどのテキストデータのクラスタリングでよく使われるモデル

##  潜在ディリクレ配分
トピックモデルの手法


# 8.4.1　潜在ディリクレ配分を使ってテキスト文書を分解する



# 8.4.2　scikit-learnの潜在ディリクレ配分

In [1]:
#映画レビューデータセットのローカルファイルを使う
import tarfile
import pandas as pd
#with tarfile.open('aclImdb_v1.tar.gz', 'r:gz') as tar:
#    tar.extractall()
df=pd.read_csv('movie_data.csv',  encoding='utf-8')

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
count=CountVectorizer(stop_words='english',
                     max_df=.1,  #単語の最大文書頻度を10%
                     max_features=5000)   #出現頻度が最も高いとみなされる単語の偶数を5000個に制限する
X=count.fit_transform(df['review'].values)

In [3]:
#バッチ学習を設定することで１回のイテレーションでlda推定器が利用可能な全ての訓練データ(BoW行列)に基づいて推定を行う
from sklearn.decomposition import LatentDirichletAllocation
lda=LatentDirichletAllocation(n_components=10,
                                            random_state=123,
                                            learning_method='batch')
X_topics=lda.fit_transform(X)

In [4]:
#10種類のトピックごとに単語の重要度(5000個)を含んだ行列が含まれている
lda.components_.shape

(10, 5000)

In [5]:
n_top_words=5
feature_names=count.get_feature_names()
for topic_idx, topic in enumerate(lda.components_):
    print("Topic. %d:"  % (topic_idx+1))
    print("".join([feature_names[i]
            for i in topic.argsort()  \
                [:-n_top_words - 1:-1]]))

Topic. 1:
worstminutesawfulscriptstupid
Topic. 2:
familymotherfatherchildrengirl
Topic. 3:
americanwardvdmusictv
Topic. 4:
humanaudiencecinemaartsense
Topic. 5:
policeguycardeadmurder
Topic. 6:
horrorhousesexgirlwoman
Topic. 7:
roleperformancecomedyactorperformances
Topic. 8:
seriesepisodewarepisodestv
Topic. 9:
bookversionoriginalreadnovel
Topic. 10:
actionfightguyguyscool


In [6]:
horror=X_topics[:,5].argsort()[::-1]
for iter_idx, movie_idx in enumerate(horror[:3]):
    print('\nHorror movie #%d:'  % (iter_idx+1))
    print(df['review'][movie_idx][:300],  '...')


Horror movie #1:
House of Dracula works from the same basic premise as House of Frankenstein from the year before; namely that Universal's three most famous monsters; Dracula, Frankenstein's Monster and The Wolf Man are appearing in the movie together. Naturally, the film is rather messy therefore, but the fact that ...

Horror movie #2:
Okay, what the hell kind of TRASH have I been watching now? "The Witches' Mountain" has got to be one of the most incoherent and insane Spanish exploitation flicks ever and yet, at the same time, it's also strangely compelling. There's absolutely nothing that makes sense here and I even doubt there  ...

Horror movie #3:
<br /><br />Horror movie time, Japanese style. Uzumaki/Spiral was a total freakfest from start to finish. A fun freakfest at that, but at times it was a tad too reliant on kitsch rather than the horror. The story is difficult to summarize succinctly: a carefree, normal teenage girl starts coming fac ...


In [8]:
from IPython.display import Image

In [25]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/suzukamayo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

stop = stopwords.words('english')
porter = PorterStemmer()
"""
def tokenizer(text):
    text = re.sub('<[^>]*>',f '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

def stream_docs(path):
    with open(path, 'r', encoding='utf-8') as csv:
        next(csv) # skip header
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label
"""

"\ndef tokenizer(text):\n    text = re.sub('<[^>]*>',f '', text)\n    emoticons = re.findall('(?::|;|=)(?:-)?(?:\\)|\\(|D|P)', text.lower())\n    text = re.sub('[\\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')\n    tokenized = [w for w in text.split() if w not in stop]\n    return tokenized\n\ndef stream_docs(path):\n    with open(path, 'r', encoding='utf-8') as csv:\n        next(csv) # skip header\n        for line in csv:\n            text, label = line[:-3], int(line[-2])\n            yield text, label\n"

In [7]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier

clf = SGDClassifier(loss='log', random_state=1, max_iter=1)


# 9章  機械学習の適用2 -Webアプリケーション

### Webアプリケーションでの機械学習モデルの有益な応用例

入力フォーム
検索エンジン
メディア
ショッピングサイトのレコメンデーションシステムでのスパム検出

### 訓練した機械学習モデルの現在の状態を保存する 

データストレージとしてSQLiteデータベースを使う

人気の高いWebフレームワークFlaskを使ってWebアプリケーションを開発する

機械学習アプリケーションをパブリックWebサーバーにデプロイする

# 9.1 　学習済みのscikit-learn推定器をシリアライズする

# #モデル永続化


In [5]:
import pickle
import os

#Webアプリケーションのファイルとデータの格納先となるmovieclassifier
#その下にpkl_objectsサブディレクトリを作成 
dest=os.path.join('movieclassifier',  'pkl_objects')  #シリアライズされたpythonオブジェクトをローカルドライブに保存するときに使われる
if not os.path.exists(dest):
    os.makedirs(dest)
pickle.dump(stop,open(os.path.join(dest,'stopwords.pkl'),'wb'),
                   protocol=4)
pickle.dump(clf,
                   open(os.path.join(dest,'classifier.pkl'),'wb'),
                   protocol=4)

In [20]:
'''
HashingVectorizerは学習させる必要がないため，シリアライズする必要はない，
代わりに新しいpythonスクリプトファイルを作成し，そのファイルからベクタライザを
現在のpythonセッションにインポートできるようにする

'''
from sklearn.feature_extraction.text import HashingVectorizer
import re
import os
import pickle
from pathlib import Path

#self.log_dir = os.path.join(Path().resolve(), "logdir")
#Path().resolve()
cur_dir=os.path.dirname('__file__')
stop=pickle.load(open(os.path.join(
                        cur_dir,   'pkl_objects',  'stopwords.pkl'),'rb'))

def tokenizer(text):
    text=re.sub('<[^>]*>',  '',  text)
    emoticons=re.findall('(?::|;|=)(?:-)?(?:\) | \(|D|P)',  text.lower())
    text=re.sub('[\W]+',  '',  text.lower())  \
        + ''.join(emoticons).replace('-',  '')
    tokenized=[w for w in text.split() if w not in stop]
    return tokenized

vect=HashingVectorizer(decode_error='ignore',
                                     n_features=2**21,
                                     preprocessor=None,
                                     tokenizer=tokenizer)


In [1]:
import pickle
import re
import os
from vectorizer import vect
clf=pickle.load(open(os.path.join('pkl_objects',  'classifier.pkl'), 'rb'))

In [2]:
'''
この分類器は予測したクラスラベルを整数として返す．単純なディクショナリを定義することで
label変数でそれらの整数を環状のラベルにマッピングしている
これはクラスが２つだけの単純な例だがこのディクショナリマッピングアプローチが
多クラス分類に対しても一般化される点に注意する必要がある．

'''
import numpy as np
label={0: 'negative ', 1:'positive'}
example=["I love thsi movei. It's  amazing."]
X=vect.transform(example)  #単純なデータをワードベクトルXに変換

#ロジスティック回帰の分類器のpredictメソッドを使ってクラスラベルを予測し，
#predict_probaメソッドを使ってその予測に対応する確率を取得している
#predict_probaメソッドを呼ぶことで一意なクラスラベルごとに確率値が含まれた
#配列が返される 
print('Prediction: %s\nProbability:  %.2f%%'  %\
         (label[clf.predict(X)[0]], np.max(clf.predict_proba(X))*100)) 

NotFittedError: This SGDClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.